# PipelineEndpoint Demo

In [ ]:
from azureml.pipeline.wrapper import Module, dsl
from workspace_helpers import setup_default_workspace, load_or_register_dataset
from azureml.pipeline.wrapper._pipeline_endpoint import PipelineEndpoint

In [ ]:
workspace = setup_default_workspace()

In [ ]:
experiment_name = "TestPipelineEndpoint"
module_func = Module.load(workspace, namespace='azureml', name='Select Columns in Dataset')
test_pipeline_endpoint_name = "test_pipeline_endpoint"
test_pipeline_endpoint_id = "b27eb28c-da34-4064-9c13-e515fdb2c71b"

blob_input_data = load_or_register_dataset(workspace, name='Automobile_price_data_(Raw)',
                                           datastore_name='azureml_globaldatasets',
                                           path='GenericCSV/Automobile_price_data_(Raw)')
select_columns = "{\"isFilter\":true,\"rules\":[{\"exclude\":true,\"ruleType\":\"ColumnNames\",\"columns\":" \
                 "[\"normalized-losses\"]}]}"

@dsl.pipeline(name='test-pipeline', default_compute_target='aml-compute')
def test_pipeline(_select_columns):
    module1 = module_func(
        dataset=blob_input_data,
        select_columns=_select_columns
    )
    return module1.outputs

In [ ]:
pipeline = test_pipeline(_select_columns=select_columns)
pipeline_run = pipeline.submit(experiment_name=experiment_name)

### 1. Publish
Create a PipelineEndpoint with the specified name and pipeline/pipeline_run.

#### 1.1 publish by pipeline

In [ ]:
pipeline_endpoint_published_by_pipeline = PipelineEndpoint.publish(
    workspace=workspace,
    name=test_pipeline_endpoint_name,
    set_as_default=True,
    description="description_for_test_endpoint_publish",
    pipeline=pipeline)

pipeline_endpoint_published_by_pipeline

#### 1.2 publish by pipeline_run

In [ ]:
# pipeline_run test
pipeline_endpoint_published_by_pipeline_run = PipelineEndpoint.publish(
    workspace=workspace,
    name=test_pipeline_endpoint_name,
    set_as_default=True,
    description="description_for_test_endpoint_publish",
    pipeline=pipeline_run
)

pipeline_endpoint_published_by_pipeline_run

### 2. Get
Get the PipelineEndpoint by name or ID

#### 2.1 get by id

In [ ]:
endpoint_got_by_id = PipelineEndpoint.get(workspace=workspace, id=test_pipeline_endpoint_id)

endpoint_got_by_id

#### 2.2 get by name

In [ ]:
endpoint_got_by_name = PipelineEndpoint.get(workspace=workspace, name=test_pipeline_endpoint_name)

endpoint_got_by_name

### 3. Submit
Submit default version of endpoint.

In [ ]:
test_submit_experiment_name = "test_submit_by_pipeline_endpoint"
pipeline_endpoint = PipelineEndpoint.get(workspace=workspace, id=test_pipeline_endpoint_id)

In [ ]:
paramesters = {'_select_columns': '{"isFilter":true,"rules":[{"exclude":true,"ruleType":"ColumnNames","columns":["normalized-losses"]}]}'}
run_submitted_by_pipeline_endpoint = pipeline_endpoint.submit(experiment_name=test_submit_experiment_name,
                                                             parameters = paramesters)

In [ ]:
run_submitted_by_pipeline_endpoint.tags["azureml.pipelineid"]

In [ ]:
run_submitted_by_pipeline_endpoint

### 4. List
List active PipelineEndpoints in the current workspace.

In [ ]:
pipeline_endpoints_list = PipelineEndpoint.list(workspace=workspace, active_only=True)

In [ ]:
pipeline_endpoints_list

In [ ]:
print("pipeline endpoints name list:")
[endpoint.name for endpoint in pipeline_endpoints_list]

### 5. Enable & Disable

In [ ]:
pipeline_endpoint.enable()
print(pipeline_endpoint.status)

In [ ]:
pipeline_endpoint.disable()
print(pipeline_endpoint.status)

In [ ]:
pipeline_endpoint.enable()
print(pipeline_endpoint.status)

### 6. List pipelines
List of PublishedPipelines in PipelineEndpoint, with format {version: pipeline}

active_only: Whether to return only active pipelines.

In [ ]:
pipelines_list = pipeline_endpoint.list_pipelines(active_only=True)

In [ ]:
pipelines_list

### 7. Set default version

In [ ]:
default_version = '0'
print('default version BEFORE set default version:', pipeline_endpoint.default_version)
pipeline_endpoint.set_default_version(version=default_version)
print('default version AFTER set default version:', pipeline_endpoint.default_version)

# Finish
Disable published pipeline

In [ ]:
version_list = list(pipelines_list.keys())
pipelines_list[version_list[0]].disable()
pipelines_list[version_list[1]].disable()